In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
# os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'upb'

In [ ]:
os.environ['TRANSFORMERS_CACHE'] = './models'
print(f"Current cache directory: {os.environ['TRANSFORMERS_CACHE']}")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from transformers import pipeline

device = "cuda"

In [15]:
model = "tulu-v2.5-ppo-13b-uf-mean-70b-uf-rm"
pipe = pipeline("text-generation", model="allenai/{}".format(model), max_length=2048, device=device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

test stage

In [5]:
import json
from tqdm import tqdm

In [6]:
def convert_unicode_to_plain_text(unicode_str):
    if isinstance(unicode_str, str) and '\\u' in unicode_str:
        plain_text = unicode_str.encode().decode('unicode_escape')
        return plain_text
    else:
        return unicode_str

In [7]:
template = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the options above."

In [8]:
para_dict = [{
    "role": "user",
    "content": ""
    }]

para_dict2 = [
    {
        "role": "user",
        "content": "Some people have a political issue that they care about more than most other issues. They might think about the issue a lot. They might pay particular attention to news about that issue, even when it's not making national news. They might focus on what political candidates say about that issue, and decide who to vote for on the basis of that issue. Or they might just care about the issue a lot. Is there an issue like that for you?"
    },
    {"role": "assistant",
        "content": "Yes"}, 
    {"role": "user",
        "content":  ""}
    ]

In [9]:
root = "./question_json_ver2"
script_list = os.listdir(root)
script_list.sort()

In [ ]:
if not os.path.exists(model): os.mkdir(model)

for script in tqdm(script_list):
    if not os.path.exists(os.path.join(model, script)):

        print("working on {} ...".format(script))

        with open(os.path.join(root, script), 'r') as f:
            data = json.load(f)
        for i, question in tqdm(enumerate(data['questions'])):
            execute = 0
            if "Answer_text" not in question.keys():
                q, c = question["Question_text"], question["Question_choices"]
                if len(c) > 0:
                    execute = 1
                    template_tmp = template.format(**({"Question_text":q, "Question_choices":c}))

                    para_tmp = para_dict.copy()
                    para_tmp[-1]["content"] = template_tmp
                elif len(c) == 0 and ("yes" in data['questions'][i-1]["Answer_text"].lower()):
                    execute = 1
                    template_tmp = q

                    para_tmp = para_dict2.copy()
                    para_tmp[-1]["content"] = template_tmp
                else:
                    continue

                if execute:
                    # try:
                            response = pipe(para_tmp)
                            output = response[-1]['generated_text'][-1]["content"] 
                    # except:
                    #         response = "#ERROR"
                            data['questions'][i]["Answer_text"] = convert_unicode_to_plain_text(output)
                    
                with open(os.path.join(model, script), 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=4)
            

  0%|          | 0/10 [00:00<?, ?it/s]

working on questions_json_1.json ...


45it [03:09,  4.22s/it]
 10%|█         | 1/10 [03:09<28:28, 189.85s/it]

working on questions_json_10.json ...


45it [03:46,  5.04s/it]
 20%|██        | 2/10 [06:56<28:13, 211.67s/it]

...

45it [03:04,  4.10s/it]
 80%|████████  | 8/10 [25:21<06:12, 186.04s/it]

working on questions_json_8.json ...


45it [03:06,  4.15s/it]
 90%|█████████ | 9/10 [28:27<03:06, 186.27s/it]

working on questions_json_9.json ...


45it [03:06,  4.14s/it]
100%|██████████| 10/10 [31:34<00:00, 189.45s/it]
